# Process a language tab to yaml v. 2.0

This notebook collects language tabs from the SAPhon [Tupian Nasal Typology Input](https://docs.google.com/spreadsheets/d/1dvXFvLIV4y84CglgjAl-ZVb09IuGazs1SzFO_UJpmnI/edit#gid=1164878023) spreadsheet and creates version 2.0 yaml output.

In [ ]:
import spreadsheet
import os, re, sys
import requests
from pathlib import Path
import pandas as pd
import yaml
import json
import math

downloads = Path.home() / 'Downloads'
langdir = Path('./newlangs/')
langdir.mkdir(parents=True, exist_ok=True)
yamldir = Path('../langs')

## Get Tupian input spreadsheet lang tabs

Collect the language tabs from the input spreadsheet into a dataframe, one row per lang tab.

In [ ]:
ssdf = pd.DataFrame.from_records(list(spreadsheet.langsheets.values()))
ssdf['tabname'] = list(spreadsheet.langsheets.keys())
ssdf['yaml'] = ssdf['short'] + '.yaml'
ssdf = ssdf[ssdf['include']].reset_index(drop=True).drop('include', axis='columns')
assert(~ssdf['gid'].duplicated().any())
assert(~ssdf['tabname'].duplicated().any())
ssdf

## Download `.tsv` files (optional)

The next cell is optional to download all lang tabs from the spreadsheet. Set `do_download` to `True` and execute the cell to do this task. For active work on a lang tab this step is not necessary and time-consuming.

In [ ]:
do_download = False
if do_download:
    for row in ssdf[ssdf['include']].itertuples():
        r = requests.get(f'{spreadsheet.puburl}/pub?gid={row.gid}&single=true&output=tsv')
        r.encoding = 'utf-8'
        with open(langdir / f'{row.short}.tsv', 'w', encoding='utf-8') as out:
            out.write(r.text)

## Function definitions

Functions used to create version 2.0 yaml, work in progress.

In [ ]:
#langre = re.compile(
#    r'''
#    (?P<name>[^\[]+)
#    (?P<iso>\[[^\]]+\])?
#    ''',
#    re.VERBOSE
#)

def _clean(s):
    '''
    Clean string of extraneous markup.
    '''
    if s != None:
        s = s.strip().strip('{').strip('}').strip('[').strip(']').strip()
    return s

def split_morph_ids(mids):
    morphs = []
    if mids is None or mids == '' or mids.lower().strip() == 'none':
        return morphs
    for mid in spreadsheet.parse_with_delims(mids):
        try:
            d = {
                'morpheme_id': mid[0],
                'morpheme_type': mid[1],
                'underlying_form': mid[2],
                'surface_forms': spreadsheet.parse_with_delims(mid[3].strip('{').strip('}')),
                'gloss': mid[4]
            }
        except Exception as e:
            msg = f'Could not parse morpheme ids: {mid}: {e}'
            raise RuntimeError(msg)
        morphs.append(d)
    return morphs

def alloprocs(allos, procs, phoneme):
    '''
    Return zipped allophones and processes extracted from allophone list.
    '''
    allolist = [_clean(a) for a in allos.split(',')]
    proclist = [_clean(p) for p in procs.split(',')]
    mapping = {a: [] for a in allolist}

    try:
        assert(len(proclist) > 0)
    except AssertionError:
        sys.stderr.write(f'Invalid entry for phoneme {phoneme}, must have at least one allophone and process\n') 
    if (len(allolist) == 1):
        mapping[allolist[0]] = proclist
    else:
        for proc in proclist:
            m = spreadsheet.procre.match(proc)
            pgd = m.groupdict()
            try:
                phone = pgd['phone'].replace('-', '')
            except AttributeError:
                sys.stderr.write(f'Process {proc} must have an allophone prefix\n')
                continue
            procname = pgd['procsubtype'] if pgd['tag'] is None else pgd['tag'] + pgd['procsubtype']
            try:
                assert(phone in mapping.keys())
                mapping[phone].append(procname)
            except AssertionError:
                sys.stderr.write(f'{proc} does not map to an allophone\n')
    for a, proclist in mapping.items():
        if a == phoneme:
            try:
                assert(proclist == [])
            except AssertionError:
                if proclist == ['preservation']:
                    pass
                else:
                    sys.stderr.write(f'Identity allophone {a} should not have a process\n')
        else:
            try:
                assert(len(proclist) > 0)
            except AssertionError:
                sys.stderr.write(f'Non-identity allophone {a} must name at least one process\n')
    return [{'allophone': k, 'processnames': v} for k, v in mapping.items()]
 

def tsv2newyaml(tsvfile, v1):
    '''
    Make a new YAML dict from a Tupian input spreadsheet tab.
    '''
    tsvlang = spreadsheet.read_lang(tsvfile)
    natclasses, flatnatclasses, catsymb = spreadsheet.check_natclasses(tsvlang)
    allophones, alloprocs = spreadsheet.check_allophones(tsvlang, flatnatclasses)
    morph_id_map = spreadsheet.check_morpheme_ids(tsvlang)
    spreadsheet.check_procs(tsvlang, flatnatclasses, morph_id_map, catsymb, alloprocs)
    # TODO: remainder should be per-doc (synthesis, ref)
    doc = tsvlang['synthesis']
    name = _clean(doc['lang'])
    if '[' in name:
        sys.stderr.write(f'Language name for {name} should be cleaned up.\n')
    try:
        notes = doc['notes']
    except KeyError:
        notes = 'None'
    try:
        glottocode = _clean(doc['glottocode'])
    except KeyError:
        sys.stderr.write(f'Lang {name} missing glottocode field.\n')
        glottocode = TODO
    try:
        altnames = [_clean(c) for c in doc['altnames'].split(',')]
    except KeyError:
        sys.stderr.write(f'Lang {name} missing alternate_names field.\n')
        altnames = [TODO]
    try:
        iso_codes = [_clean(c) for c in doc['iso_codes'].split(',')]
    except KeyError:
        sys.stderr.write(f'Lang {name} missing iso_codes field.\n')
        iso_codes = [TODO]
    if v1 == {}:
        v1 = {
            'short_name': 'TODO',
            'alternate_names': 'TODO',
            'family': 'TODO',
            'countries': 'TODO',
            'coordinates': 'TODO',
            'tone': 'TODO',
            'laryngeal_harmony': 'TODO'
        }
    try:
        sdoc = {
            'doctype': 'synthesis',
            'name': name,
            'glottolog_code': glottocode,
            'short_name': v1['short_name'],
            'alternate_names': altnames,
            'iso_codes': iso_codes,
            'synthesis': doc['synthesis'],
            'natural_classes': [{'symbol': nc[0], 'members': nc[1:]} for nc in natclasses['synthesis']],
            'glottolog_codes': ['TODO'], # TODO: new, need to be added by hand
            'family': v1['family'],
            'countries': v1['countries'],
            'coordinates': v1['coordinates'],
            'morphemes': split_morph_ids(doc['morph_ids']),
            'phonemes': phonlist(allophones['synthesis']),
            'processdetails': doc['processes'],
         # TODO: following from v1 and not mentioned in new YAML draft
#!        'allophones': v1['allophones'],
#!        'nasal_harmony': v1['nasal_harmony'],
            'tone': v1['tone'],
            'laryngeal_harmony': v1['laryngeal_harmony'],
            'notes': notes
        }
    except Exception as e:
        sys.stderr.write(f'Problem with doc {doc}.\n')
        raise e
    # Filter None values out of list values.
#!    listflds = (
#!        'alternate_names', 'iso_codes', 'glottolog_codes', 
#!        'countries', 'coordinates', 'natural_classes',
#!        'morphemes', 'phonemes', 'processes',
#!        'triggers', 'notes'
#!    )
#!    for fld in listflds:
#!        sdoc[fld] = [v for v in sdoc[fld] if v is not None]
    return (sdoc, tsvlang, allophones)

def proclist_old(processes):
    '''
    Return a `processdetails` list from spreadsheet `processes` section.
    '''
    deets = []
    for proc in processes:
        procname = proc['proc_name'] if not '-' in proc['proc_name'] else proc['proc_name'][proc['proc_name'].index('-')+1:]
        deet = {
            'processname': procname,
            'processtype': proc['proc_type'],
            'description': proc['description'],
            'optionality': proc['optionality'],
            'directionality': proc['directionality'],
            'alternation_type': proc['alternation_type']
        }
        for fld in 'undergoers', 'triggers':
            if proc[fld] == 'NA':
                deet[fld] = [['NA'], {'type': 'TODO', 'positional_restrictions': 'TODO'}]
            elif isinstance(proc[fld], dict):
                data = proc[fld][fld]
            else:
                print(f'PROC: {proc}')
                data = proc[fld][0][fld]
            try:
                deet[fld] = [
                    [_clean(f) for f in data.split(',')],
                    {
                        'type': 'TODO',
                        'positional_restrictions': 'TODO'
                    }
                ]
            except:
                print(f'failed for {fld}: "{proc[fld]}"')
        for new, old in ('transparent', 'transparencies'), ('opaque', 'opacities'):
            deet[new] = proc[old][old]
        deets.append(deet)
    return deets

def _clean_procname(p):
    if p is None:
        return 'TODO: got None'
    else:
        return _clean(p if '-' not in p else p[p.index('-')+1:])

def phonlist(allophones):
    '''
    Return a `phonemes` list from an `allophone` set.
    '''
    phonemes = {}
    for pset in allophones:
        d = {}
        if len(pset) == 5:
            seg = pset[0]
            envs = {'phoneme': f'STRING MAPPING: "{pset}"'}
            print(f'STRING MAPPING: "{pset}"')
        elif len(pset) == 4:
            phoneme, allos, _psetenv, proc = pset
            allophones = alloprocs(allos, proc, phoneme)
            envs = []
            for env in spreadsheet.parse_env(pset[2]):
                try:
                    env['allophones'] = allophones
                except:
                    print(env)
                envs.append(env)
        elif len(pset) == 2:
            phoneme, allos = pset
            envs = [{
                'preceding': '', # TODO: NA or other empty value here?
                'following': '', # TODO: NA or other empty value here?
                'allophones': [
                    {
                        'processnames': [],
                        'allophone': _clean(a)
                    } for a in allos.split(',')
                ]
            }]
        else:
            sys.stderr.write(f'Unexpected phoneme set length for "{pset}".')
            continue
        try:
            phonemes[phoneme]['environments'] += envs
        except KeyError:
            phonemes[phoneme] = {'phoneme': phoneme, 'environments': envs}
    # TODO: sort phonemes
    return list(phonemes.values())

def yaml2newyaml(v1):
    '''
    Copy a version 1 YAML dict into version 2.
    '''
    sdoc = {
        'doctype': 'synthesis',
        'name': v1['name'],
        'glottolog_name': v1['name'], # TODO: new, check by hand
        'short_name': v1['short_name'],
        'alternate_names': v1['alternate_names'],
        'iso_codes': v1['iso_codes'],
        'glottolog_codes': [], # TODO: new, need to be added by hand
        'family': v1['family'],
        'countries': v1['countries'],
        'coordinates': v1['coordinates'],
        'natural_classes': [], # TODO: new
        'morphemes': [], # TODO: new?
        'phonemes': [{'phoneme': p} for p in v1['phonemes']],
        'processdetails': [], # TODO: new?
        'triggers': [], # TODO: new?
        'transparent': [], # TODO: new?, include?
        'opaque': [], # TODO: new?, include?
         # TODO: following from v1 and not mentioned in new YAML draft
        'allophones': v1['allophones'],
        'nasal_harmony': v1['nasal_harmony'],
        'tone': v1['tone'],
        'laryngeal_harmony': v1['laryngeal_harmony'],
        'notes': v1['notes'], # TODO: not mentioned in new YAML draft
    }
    # Filter None values out of list values.
    listflds = (
        'alternate_names', 'iso_codes', 'glottolog_codes', 
        'countries', 'coordinates', 'natural_classes',
        'morphemes', 'phonemes', 'processdetails',
        'triggers', 'notes'
    )
    for fld in listflds:
        sdoc[fld] = [v for v in sdoc[fld] if v is not None]
    return sdoc

TODO = 'TODO'

def ss2refdoc(lang):
    pass

def ss2synthdoc(lang):
    '''
    Produce a synthesis yaml doc from a lang from the input spreadsheet.
    '''
    synth = lang['synthesis']
    langm = re.match(langre, synth['lang'])
    yd = {
        'doctype': 'synthesis',
        'name': langm['name'].strip(),
        'short_name': TODO,
        'alternate_names': TODO,
        'iso_codes': langm['iso']
    }
    return yd

In [ ]:
# This is just a debugging cell

sample = '''{pá, stem, pá, má, US}, {potá, stem, potá, motá, US}, {ɣʷá, postposition, ɣʷá, ŋʷá, US}, {ɣʷaré, postposition, ɣʷaré, ŋʷaré, US}, {tɨ́, suffix, tɨ́, nɨ́, US}, {pɨ́, suffix, pɨ́, mɨ́, US}, {kʷéra, stem, kʷéra, ŋʷéra, US}, {ɣʷivé, stem, ɣʷivé, ŋʷivé, US}, {pukú, stem, pukú, mukú, US}, {ɣʷasú, stem, ɣʷasú, ŋʷasú, US}, {kʷé, der.affix, kʷé, ŋʷé, US}, {pe, postposition, pe, {mẽ, ʋe, ʋ̃ẽ}, US}, {ʔó, der.affix, ʔó, ʔṍ, US}, {sé, der.affix, sé, sẽ́, US}, {ɣʷi, postposition, ɣʷi, ɣ̃ʷ̃ĩ, conjunction}, {ke, stem, ke, kẽ, modifier}, {ko, stem, ko, kõ, pronoun}, {ku, stem, ku, kũ, article}, {la, stem, la, l̃ã, article}, {na...i, stem, na...i, nã...ĩ, modifier}, {pa, stem, pa, pã, clause marker}, {ʃa, postposition, ʃa, ʃã, NA}, {ta, stem, ta, tã, modifier}, {tei, stem, tei, tẽĩ, modifier}, {ʋa, stem, ʋa, ʋ̃ã, nominalizer}, {ʋo, stem, ʋo, ʋ̃õ, conjunction}, {xa, stem, xa, xã, coordinator}, {pikó, stem, pikó, {pi, pĩ}, clause marker}, {rexé, postposition, rexé, {re, r̃ẽ}, NA}, {riré, postposition, riré, {ri, r̃ĩ}, conjunction} '''
#split_morph_ids(tsvlang['synthesis']['morph_ids'])
print(split_morph_ids(sample))

In [ ]:
# This is just a debugging cell

samplestrs = [
#    'p, p',
#    'e, {e, ɛ}, @, ɛ-lowering',
#    's, {s, tʃ, ʃ}, {_i, i_}, {tʃ-fortition, tʃ-palatalization, ʃ-palatalization}',
#    't, n, _+Ṽ, XMP=LN:t',
#    "e, {eː, ɛː}, $'_, {eː-lengthening, ɛː-lowering, ɛː-lengthening}",
#    'o, {o,ŏ}, _{h, ɾ}o, ŏ-vowel shortening',
    "o, ɑ, {$_$'{ʔɔ, hɔ},w_}, lowering",
    'a, {ə̃, ã}, {_{N, ND}, N_}, {ə̃-LN:V, ã-LN:V}'
]
for s in samplestrs:
    pset = spreadsheet.split_outside_delims(s)
    if len(pset) == 4:
        print(pset)
        phoneme, allos, env, proc = pset
        mapping = alloprocs(allos, proc, phoneme)
        print(mapping)

## Read `.tsv` and v1 `.yaml` files

Download, read, and process lang tabs (and existing v. 1 yaml files, if they exist). Set one or more tab indexes in `rng` to be checked for errors. Set `use_cached` to `True` if you want to use a previously-downloaded `.tsv` file instead of downloading from the input spreadsheet.

In [ ]:
nosynthesis = [
    12,
    21,
    25,
    31,
    34,
    36,
    42,
    43,
    46,
    47,
    49, # Siriono
    50,
    53, # Tapirapé
    56,
    62,
    64,
    66
]
# investigate
# 16 phonemes/allophones phonlist()
# 21, 28, 34, 36, 42, 44, 51, 62, 64 proc details
# 66 bad parse at high level

In [ ]:
use_cached = False
langs = {}
#rng = [0, 2, 3, 5, 20, 29, 37, 40] # Indexes that have no errors so far
rng =  [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9]
rng += [    11,                     17, 18, 19]  # List of one or more tab indexes to process
rng += [20,         23, 24,         27,     29]
rng += [30,     32, 33,                 38    ]
rng += [40, 41,                         48, 49]
rng += [            53,     55,         58, 59]
rng += [60, 61,     63,               ] # 67 is out of bounds
rng = [n for n in range(67)] # if n not in rng]
rng = [e for e in rng if e not in nosynthesis]
print(f'Working on {len(rng)} lang tabs')
for row in ssdf.iloc[rng].itertuples():
    if (yamldir / row.yaml).exists():
        with open(yamldir / row.yaml, 'r', encoding='utf-8') as fh:
            v1docs = list(yaml.safe_load_all(fh))
            if math.isnan(v1docs[0]['coordinates'][0]['elevation_meters']):
                v1docs[0]['coordinates'][0]['elevation_meters'] = None
        v1synth = yaml2newyaml(v1docs[0])
    else:
        v1synth = {}
    
    tsvfile = langdir / f'{row.short}.tsv'
    if use_cached is not True or not tsvfile.exists():
        print(f"Requesting '{row.tabname}' lang tab from index {row.Index} and caching at {tsvfile}.")
        r = requests.get(f'{spreadsheet.puburl}/pub?gid={row.gid}&single=true&output=tsv')
        r.encoding = 'utf-8'
        with open(tsvfile, 'w', encoding='utf-8') as out:
            # Replace Windows CRLF with Unix LF
            text = r.content.replace(b'\r\n', b'\n').decode('utf8')
            out.write(text)
    else:
        print(f"Reading from cached file {tsvfile} ({row.Index}).")
    try:
        v2synth, tsvlang, allophones = tsv2newyaml(tsvfile, v1synth)
    except Exception as e:
        v2synth = {}
        sys.stderr.write(f'ERROR: spreadsheet tab {row.tabname} failed.\n{e}\n\n')
        raise e
    langs[row.short] = {
        'v1synth': v1synth,
        'v2synth': v2synth,
        'tsv': tsvlang['synthesis']
    }
    try:
        with open(langdir / 'json' / f'{row.short}.json', 'w', encoding='utf8') as out:
            json.dump([langs[row.short]['v2synth']], out, indent=2, ensure_ascii=False)
        print(f'Dumped json {row.short}.json')
    except Exception as e:
        sys.stderr.write(f'Failed to dump json {row.short}.json.\n')